In [1]:
import gc
import glob
import os
import json
import matplotlib.pyplot as plt
import pprint

import numpy as np
import pandas as pd

from joblib import Parallel, delayed
from tqdm import tqdm
from PIL import Image
from sklearn.metrics import cohen_kappa_score, make_scorer
from sklearn.model_selection import GridSearchCV, cross_val_score, train_test_split
import scipy as sp

from sklearn.model_selection import train_test_split, GridSearchCV

from functools import partial
from collections import Counter

import random
import math

# 评价函数

In [82]:
#%% 评价函数 Metric used for this competition 
# (Quadratic Weigthed Kappa aka Quadratic Cohen Kappa Score)
def metric(y1,y2):
    return cohen_kappa_score(y1, y2, weights = 'quadratic')


# Make scorer for scikit-learn
scorer = make_scorer(metric)

# Cross验证函数

In [83]:
from sklearn.model_selection import StratifiedKFold

#
def split_score(model, x_train, y_train, x_test, n=10):
    
    y_pre = np.zeros(x_train.shape[0])
    oof_test = np.zeros((x_test.shape[0], n))
    
    kfold = StratifiedKFold(n_splits=n, random_state=4)
    aaa = 0
    
    for train_index, test_index in kfold.split(x_train, y_train):
        model.fit(x_train.iloc[train_index], y_train.iloc[train_index])
        y_pre[test_index] = model.predict(x_train.iloc[test_index])
        test_pred = model.predict(x_test)
        oof_test[:, aaa] = test_pred
        
        print(aaa)
        aaa += 1
    
#    score = metric(y_pre, y)
    print("{}折后的Kappa加权得分为:带补充".format(n))
    
    return y_pre, oof_test

#
def fix_y(y, coef):
    y_fix = np.copy(y)
    for i, pred in enumerate(y_fix):
        if pred < coef[0]:
            y_fix[i] = 0
        elif pred >= coef[0] and pred < coef[1]:
            y_fix[i] = 1
        elif pred >= coef[1] and pred < coef[2]:
            y_fix[i] = 2
        elif pred >= coef[2] and pred < coef[3]:
            y_fix[i] = 3
        else:
            y_fix[i] = 4    
    return y_fix

# 
def _kappa_loss(y, y_true, coef):
    y_fix = np.copy(y)
    for i, pred in enumerate(y_fix):
        if pred < coef[0]:
            y_fix[i] = 0
        elif pred >= coef[0] and pred < coef[1]:
            y_fix[i] = 1
        elif pred >= coef[1] and pred < coef[2]:
            y_fix[i] = 2
        elif pred >= coef[2] and pred < coef[3]:
            y_fix[i] = 3
        else:
            y_fix[i] = 4
            
    loss = metric(y_fix, y_true)
    return -loss

# 寻找分类的最佳参数
def search_coef(x1, x2):
    loss_partial = partial(_kappa_loss, x1, x2)
    initial_coef = [1.55, 2.05, 2.5, 3]
    coef = sp.optimize.minimize(loss_partial, initial_coef, method= 'nelder-mead')

    return coef

# 读取数据、划分验证集

In [84]:
df_train  = pd.read_csv('train.csv')
x = df_train.copy()

#读取唯一的RescuerID
RescuerID = set(df_train['RescuerID'].unique())

#随机生成RescuerID
j_test = random.sample(RescuerID, int(len(RescuerID)*0.17))
j_train = RescuerID - set(j_test)

df_test = df_train[df_train['RescuerID'].isin(j_test)]
df_train = df_train[df_train['RescuerID'].isin(j_train)]

train = df_train.copy()
test  = df_test.copy()

In [85]:
labels_breed = pd.read_csv('breed_labels.csv')
labels_state = pd.read_csv('color_labels.csv')
labels_color = pd.read_csv('state_labels.csv')

In [86]:
#%% 删除异常值
cul_drop = ['375905770', 'da8d4a273', '27e74e45c', '7b5bee232', '0327b8e94']
df_train = df_train[~df_train['PetID'].isin(cul_drop)]

# 提取 sentiment 的特征

In [87]:
def extract_sentiment_feature(i, x):    
#    feature_sentiment = pd.DataFrame(columns=['PetID', 'token', 'sentence_magnitude', 'sentence_score','document_magnitude', 'document_score'])
    feature_sentiment = pd.DataFrame()

    if x == 'train':
        set_file = 'train'
    else:
        set_file = 'train' 
        
    file_name = '{}_sentiment/{}.json'.format(set_file, i)
    try:
        with open(file_name, 'r', encoding='utf-8') as f:
            sentiment_file = json.load(f)

            token = [x['name'] for x in sentiment_file['entities']]
            token = ' '.join(token)

            sentences_sentiment = [x['sentiment'] for x in sentiment_file['sentences']]
            sentences_sentiment = pd.DataFrame.from_dict(
                sentences_sentiment, orient='columns')

            
            docementSentiment_magnitude = sentiment_file['documentSentiment']['magnitude']
            documentSentiment_score     = sentiment_file['documentSentiment']['score']
            
            new = pd.DataFrame(
                    {'PetID'    : i, 

                    'magnitude_sum' : sentences_sentiment['magnitude'].sum(axis=0),
                    'score_sum'     : sentences_sentiment['score'].sum(axis=0),
                    'magnitude_mean': sentences_sentiment['magnitude'].mean(axis=0),
                    'score_mean'    : sentences_sentiment['score'].mean(axis=0),
                    'magnitude_var' : sentences_sentiment['magnitude'].var(axis=0),
                    'score_var'     : sentences_sentiment['score'].var(axis=0),

                    'document_magnitude'  : [docementSentiment_magnitude], 
                    'document_score'      : [documentSentiment_score]})  
            feature_sentiment = feature_sentiment.append(new)
    except:
        print('{}没找到'.format(file_name))
    
    for each in feature_sentiment.columns:
        if each not in ['PetID','token']:
            feature_sentiment[each] = feature_sentiment[each].astype(float)

    return feature_sentiment

#%%
train_feature_sentiment = Parallel(n_jobs=8, verbose=1)(
        delayed(extract_sentiment_feature)(i, 'train') for i in train.PetID)
train_feature_sentiment = [x for x in train_feature_sentiment]
train_feature_sentiment = pd.concat(train_feature_sentiment, ignore_index=True, sort=False)

test_feature_sentiment = Parallel(n_jobs=8, verbose=1)(
        delayed(extract_sentiment_feature)(i, 'test') for i in test.PetID)
test_feature_sentiment = [x for x in test_feature_sentiment]
test_feature_sentiment = pd.concat(test_feature_sentiment, ignore_index=True, sort=False)

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    3.8s
[Parallel(n_jobs=8)]: Done 1592 tasks      | elapsed:    8.7s
[Parallel(n_jobs=8)]: Done 5092 tasks      | elapsed:   20.3s
[Parallel(n_jobs=8)]: Done 9992 tasks      | elapsed:   35.9s
[Parallel(n_jobs=8)]: Done 12427 out of 12427 | elapsed:   43.5s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done 268 tasks      | elapsed:    0.9s
[Parallel(n_jobs=8)]: Done 2368 tasks      | elapsed:    7.7s
[Parallel(n_jobs=8)]: Done 2566 out of 2566 | elapsed:    8.4s finished


# 提取 metadata 的特征

In [35]:
#%% 提取 metadata 的特征
#file_name = 'train_metadata/000a290e4-1.json'
#f = open(file_name, 'r')
#metadatafile = json.load(f)
def extract_metadata_feature(i, x):
    feature_metadata = pd.DataFrame()
    if x == 'train':
        set_file = 'train'
    else:
        set_file = 'train'
        
    metadata_filenames = sorted(glob.glob('{}_metadata/{}*.json'.format(set_file, i)))
    if len(metadata_filenames) > 0:
        feature_metadata_sub = pd.DataFrame()
        for ff in metadata_filenames:
            with open(ff, 'r', encoding='utf-8') as f:
                file = json.load(f)
                #label
                if 'labelAnnotations' in file:
                    file_annots = file['labelAnnotations'][:1]
                    file_top_score = np.asarray([x['score'] for x in file_annots]).mean()
                    file_top_desc = [x['description'] for x in file_annots]            
                else:
                    file_top_score = np.nan
                    file_top_desc = ['']
                #colors
                file_colors = file['imagePropertiesAnnotation']['dominantColors']['colors']            
                file_color_score = np.asarray([x['score'] for x in file_colors]).mean()
                file_color_pixelfrac = np.asarray([x['pixelFraction'] for x in file_colors]).mean()            
                #crops
                file_crops = file['cropHintsAnnotation']['cropHints']                
                file_crop_conf = np.asarray([x['confidence'] for x in file_crops]).mean()
                if 'importanceFraction' in file_crops[0].keys():
                    file_crop_importance = np.asarray([x['importanceFraction'] for x in file_crops]).mean()
                else:
                    file_crop_importance = np.nan

                new = pd.DataFrame(
                        {
                                'PetID'          : [i],
                                'annots_score'   : [file_top_score],
                                'color_score'     : [file_color_score],
                                'color_pixelfrac' : [file_color_pixelfrac],
                                'crop_conf'       : [file_crop_conf],
                                'crop_importance' : [file_crop_importance]})
#                                'annots_top_desc' : [' '.join(file_top_desc)]})
                feature_metadata_sub = feature_metadata_sub.append(new)
                
#        metadata_desc = feature_metadata_sub.groupby(['PetID'])['annots_top_desc'].unique()
#        metadata_desc = metadata_desc.reset_index()
#        metadata_desc['annots_top_desc'] = metadata_desc['annots_top_desc'].apply(lambda x:' '.join(x))
#        feature_metadata_sub.drop(['annots_top_desc'], axis=1, inplace=True)

        for each in feature_metadata_sub:
            if each not in ['PetID']:
                feature_metadata_sub[each] = feature_metadata_sub[each].astype(float)
        
        
        feature_metadata_sub = feature_metadata_sub.groupby(['PetID']).agg(['mean', 'sum','var'])
        feature_metadata_sub.columns = ['{}_{}'.format(c[0], c[1].upper()) for c in feature_metadata_sub.columns.tolist()]  
        feature_metadata_sub = feature_metadata_sub.reset_index()
                    
        feature_metadata = feature_metadata.append(feature_metadata_sub)
    return feature_metadata

#
#for each in 
train_feature_metadata = Parallel(n_jobs=8, verbose=1)(
        delayed(extract_metadata_feature)(i, 'train') for i in train.PetID)
train_feature_metadata = [x for x in train_feature_metadata]
train_feature_metadata = pd.concat(train_feature_metadata, ignore_index=True, sort=False)

test_feature_metadata = Parallel(n_jobs=8, verbose=1)(
        delayed(extract_metadata_feature)(i, 'test') for i in test.PetID)
test_feature_metadata = [x for x in test_feature_metadata]
test_feature_metadata = pd.concat(test_feature_metadata, ignore_index=True, sort=False)

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.


KeyboardInterrupt: 

# 加入metadata和图片尺寸的大小

In [89]:
picture_metadata  = pd.read_csv(r'C:\Users\ZHOU-JC\Desktop\petfinder-adoption-prediction\picture.csv')
picture_size      = pd.read_csv(r'C:\Users\ZHOU-JC\Desktop\petfinder-adoption-prediction\picture_size.csv')
img_features      = pd.read_csv(r'C:\Users\ZHOU-JC\Desktop\petfinder-adoption-prediction\img_features.csv')

#%% 连接sentiment和metadata和原始数据
x_train = df_train.merge(train_feature_sentiment, how='left', on='PetID')
x_train = x_train.merge(picture_metadata, how='left', on='PetID')
x_train = x_train.merge(picture_size, how='left', on='PetID')
x_train = x_train.merge(img_features, how='left', on='PetID')


y_train = x_train['AdoptionSpeed']
x_train.drop(['AdoptionSpeed'], axis=1, inplace=True)

x_test = df_test.merge(test_feature_sentiment, how='left', on='PetID')
x_test = x_test.merge(picture_metadata, how='left', on='PetID')
x_test = x_test.merge(picture_size, how='left', on='PetID')
x_test  = x_test.merge(img_features, how='left', on='PetID')


y_test = x_test['AdoptionSpeed']
x_test.drop(['AdoptionSpeed'],  axis=1, inplace=True)

# NLP

In [90]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import SparsePCA, TruncatedSVD, LatentDirichletAllocation, NMF

col_text = ['Description']

x = x_train.append(x_test).reset_index()
x = x[['Description', 'PetID']]

n_components = 50

x[col_text] = x[col_text].fillna('MISSING')
text_features = []


for i in  ['Description']:
    svd_ = TruncatedSVD(n_components=n_components)
        
    tfv = TfidfVectorizer(min_df=2,  max_features=None,
                          strip_accents='unicode', analyzer='word', token_pattern=r'(?u)\b\w+\b',
                          ngram_range=(1, 3), use_idf=1, smooth_idf=1, sublinear_tf=1)
    
    svd_ = TruncatedSVD(
    n_components=n_components, random_state=1337)
    
    tfidf_col = tfv.fit_transform(x.loc[:, i].values)

    
    svd_col = svd_.fit_transform(tfidf_col)
    svd_col = pd.DataFrame(svd_col)
    svd_col = svd_col.add_prefix('SVD_{}_'.format(i))


    text_features.append(svd_col)    
    
    x.drop(i, axis=1, inplace=True)
    
# Combine all extracted features:
text_features = pd.concat(text_features, axis=1)

# Concatenate with main DF:
x = pd.concat([x, text_features], axis=1)

x_train = x_train.merge(x, how='left', on='PetID')
x_test  = x_test.merge(x, how='left', on='PetID')


# 情感分析

In [91]:
from textblob import TextBlob

x = x_train.append(x_test)
x = x[['PetID', 'Description']]

x['Description'] = x['Description'].fillna("Missing")
x['Description'] = x['Description'].apply(lambda x:TextBlob(x))

x['polarity']     = x['Description'].apply(lambda x:x.sentiment[0])
x['subjectivity'] = x['Description'].apply(lambda x:x.sentiment[1])

#对情感进行分箱
bin=[-2,0,0.3,2]
x['polarity'] = pd.cut(x['polarity'], bins=bin, labels=range(3))
x['polarity'] = x['polarity'].astype(np.int32)

x_train = x_train.merge(x[['PetID', 'polarity']], how='left', on='PetID')
x_test  = x_test.merge(x[['PetID', 'polarity']], how='left', on='PetID')

# 增加新的特征

In [92]:
#是否是免费的
x_train['IsFree'] = x_train['Fee'].apply(lambda x:True if x == 0 else False)
x_test['IsFree']  = x_test['Fee'].apply(lambda x:True if x == 0 else False)

#年龄（按年）
x_train['Year'] = x_train['Age'].apply(lambda x:math.floor(x/12))
x_test['Year']  = x_test['Age'].apply(lambda x:math.floor(x/12))

#年龄分箱,按频划成5份
x = x_train.append(x_test)
x['Age_qcut'] = pd.qcut(x['Age'], 5,  duplicates='drop')
x['Age_qcut'] = pd.factorize(x['Age_qcut'])[0]
x_train = x_train.merge(x[['PetID','Age_qcut']], how='left', on='PetID')
x_test  = x_test.merge(x[['PetID','Age_qcut']], how='left', on='PetID')


#血缘的种类
x = x_train.append(x_test).reset_index()
Breed1_count = x.groupby('Breed1').size().to_frame('Breed1_count').reset_index()
x_train = x_train.merge(Breed1_count, how='left', on='Breed1')
x_test  = x_test.merge(Breed1_count, how='left', on='Breed1')

#是否稀有
a = x['Breed1'].value_counts().sort_values(ascending = False).cumsum()/len(x)
rare1_index = a[a > 0.85].index.tolist()
x_train['IsRare1'] = x_train['Breed1'].isin(rare1_index).apply(lambda x:True if x == True else False)
x_test['IsRare1']  = x_test['Breed1'].isin(rare1_index).apply(lambda x:True if x == True else False)
rare2_index = a[a > 0.72].index.tolist()
x_train['IsRare2'] = x_train['Breed1'].isin(rare2_index).apply(lambda x:True if x == True else False)
x_test['IsRare2']  = x_test['Breed1'].isin(rare2_index).apply(lambda x:True if x == True else False)

#是否常见
x_train['Is_COMMON'] = x_train['Breed1'].apply(lambda x:True if (x == 265 or x == 307 or x == 266) else False)
x_test['Is_COMMON']  = x_test['Breed1'].apply(lambda x:True if (x == 265 or x == 307 or x == 266) else False)

#照片分箱
bin=[-0.5,0.5,1.5,4.5,1000]
x_train['Photo_cut'] = pd.cut(x_train['PhotoAmt'], bins=bin, labels=range(4)).astype(np.int32)
x_test['Photo_cut']  = pd.cut(x_test['PhotoAmt'], bins=bin, labels=range(4)).astype(np.int32)

# 是否是稀有颜色
x_train['Is_rare_color1'] = x_train['Color1'].apply(lambda x:True if x==5 or x==6 or x==7 else False)
x_test['Is_rare_color1'] = x_test['Color1'].apply(lambda x:True if x==5 or x==6 or x==7 else False)
x_train['Is_rare_color2'] = x_train['Color2'].apply(lambda x:True if x==6 else False)
x_test['Is_rare_color2'] = x_test['Color2'].apply(lambda x:True if x==6 else False)

#年龄是否小于二月
x_train['Is_less_than_2month']= x_train['Age'].apply(lambda x:True if x<3 else False)
x_test['Is_less_than_2month'] = x_test['Age'].apply(lambda x:True if x<3 else False)


# RescuerID 处理

In [93]:
#%% RescuerID 处理

df = df_train.append(df_test)
data_rescuer = df.groupby(['RescuerID'])['PetID'].size().reset_index()
data_rescuer.columns = ['RescuerID', 'RescuerID_count']
#data_rescuer['rank_Rescuer_count'] = data_rescuer['RescuerID_count'].rank(pct=True)

x_train = x_train.merge(data_rescuer, how='left', on='RescuerID')
x_test  = x_test.merge(data_rescuer, how='left', on='RescuerID')

x = x_train.append(x_test)
x['RescuerID_count_cut'] = pd.qcut(x['RescuerID_count'], 5, labels=range(4), duplicates='drop').astype(np.int32)

x_train = x_train.merge(x[['PetID', 'RescuerID_count_cut']], how='left', on='PetID')
x_test  = x_test.merge(x [['PetID', 'RescuerID_count_cut']], how='left', on='PetID')

#x_train.drop(['RescuerID_count'], axis=1, inplace=True)
#x_test.drop(['RescuerID_count'], axis=1, inplace=True)

# 处理Breed

In [94]:
# 增加特征 是否有第二血统
x_train['HasSecondBreed'] = x_train['Breed2'].map(lambda x:True if x != 0 else False)
x_test['HasSecondBreed'] = x_test['Breed2'].map(lambda x:True if x != 0 else False)

train_breed_main = x_train[['Breed1']].merge(
    labels_breed, how='left',
    left_on='Breed1', right_on='BreedID',
    suffixes=('', '_main_breed'))

train_breed_main = train_breed_main.iloc[:, 2:]
train_breed_main = train_breed_main.add_prefix('main_breed_')

train_breed_second = x_train[['Breed2']].merge(
    labels_breed, how='left',
    left_on='Breed2', right_on='BreedID',
    suffixes=('', '_second_breed'))

train_breed_second = train_breed_second.iloc[:, 2:]
train_breed_second = train_breed_second.add_prefix('second_breed_')

x_train = pd.concat(
    [x_train, train_breed_main, train_breed_second], axis=1)

##############
test_breed_main = x_test[['Breed1']].merge(
    labels_breed, how='left',
    left_on='Breed1', right_on='BreedID',
    suffixes=('', '_main_breed'))

test_breed_main = test_breed_main.iloc[:, 2:]
test_breed_main = test_breed_main.add_prefix('main_breed_')

test_breed_second = x_test[['Breed2']].merge(
    labels_breed, how='left',
    left_on='Breed2', right_on='BreedID',
    suffixes=('', '_second_breed'))

test_breed_second = test_breed_second.iloc[:, 2:]
test_breed_second = test_breed_second.add_prefix('second_breed_')

x_test = pd.concat(
    [x_test, test_breed_main, test_breed_second], axis=1)

print(x_train.shape, x_test.shape)

categorical_columns = ['main_breed_BreedName', 'second_breed_BreedName']
#for i in categorical_columns:
#    x_train.loc[:, i] = pd.factorize(x_train.loc[:, i])[0]
#    x_test.loc[:,i]   = pd.factorize(x_test.loc[:, i])[0]

# 增加特征 是否纯种
x_train['True_Pure'] = False
x_train.loc[(x_train['main_breed_BreedName'] != 'Mixed Breed')&
                    ((x_train['main_breed_BreedName'] == x_train['second_breed_BreedName'])|
                   (x_train['second_breed_BreedName'].isnull())),'True_Pure'] = True

x_test['True_Pure'] = False
x_test.loc[(x_test['main_breed_BreedName'] != 'Mixed Breed')&
                    ((x_test['main_breed_BreedName'] == x_test['second_breed_BreedName'])|
                   (x_test['second_breed_BreedName'].isnull())),'True_Pure'] = True

# 是否纯种狗
x_train['Is_Pure_Dog'] = (x_train['True_Pure'] == True) & (x_train['Type'] == 1)
x_test['Is_Pure_Dog']  = (x_test['True_Pure'] == True)  & (x_test['Type'] == 1)

#删除没用特征
x_train.drop(['main_breed_BreedName', 'second_breed_BreedName', 'main_breed_Type', 'second_breed_Type'], axis=1, inplace=True)
x_test.drop(['main_breed_BreedName', 'second_breed_BreedName', 'main_breed_Type', 'second_breed_Type'], axis=1, inplace=True)

(12423, 124) (2566, 124)


# 等某些特征进行rank

In [154]:
# 对某些特征进行rank
#cols_rank = ['sentence_magnitude', 'sentence_score', 'document_magnitude','document_score']#,
#       'annots_score_MEAN', 'annots_score_SUM','color_score_MEAN', 'color_score_SUM', 'color_pixelfrac_MEAN',
#       'color_pixelfrac_SUM', 'crop_conf_MEAN', 'crop_conf_SUM','crop_importance_MEAN', 'crop_importance_SUM']

#x = x_train.append(x_test)
#x[cols_rank] = x[cols_rank].fillna(0)
#df_cols_rank = x[cols_rank].rank(pct=True).rename(columns=lambda s:'rank.'+s)
#df_cols_rank = pd.concat([df_cols_rank, x['PetID']], axis=1)

#x_train = x_train.merge(df_cols_rank, how='left', on='PetID')
#x_test =  x_test.merge(df_cols_rank, how='left', on='PetID')

# 删除部分数据\转换种类数据

In [96]:
drop_columns = ['Name', 'RescuerID', 'Description', 'PetID', 'token', 'annots_top_desc']
drop_columns = ['Name', 'RescuerID', 'Description', 'PetID']

x_train.drop(drop_columns, axis=1, inplace=True)
x_test.drop(drop_columns, axis=1, inplace=True)

x_train = x_train.fillna(-1)
x_test  = x_test.fillna(-1)


# 转换种类数据

In [97]:
#属性标签
c = ['Breed1', 'Breed2', 'Gender', 'Color1', 'Color2', 'Color3',
    'Vaccinated', 'Dewormed', 'Sterilized', 'Health', 'Quantity',
    'State', 'IsFree', 'Year', 'Age_qcut', 'IsRare1', 'IsRare2',
    'Is_COMMON', 'Photo_cut', 'Is_rare_color1', 'Is_rare_color2', 'Is_less_than_2month',
    'RescuerID_count_cut', 'HasSecondBreed', 'True_Pure', 'Is_Pure_Dog']

for each in c:
    x_train[each] = x_train[each].astype('category')
    x_test[each] = x_test[each].astype('category')

# LGB 算法

In [98]:
from lightgbm.sklearn import LGBMRegressor

model_lgb = LGBMRegressor(
        learning_rate    = 0.01,
        n_estimators     = 1000,
        max_depth        = 4,
        num_leaves       = 12 ,
        subsample        = 0.8,      #训练时采样一定比例的数据	
        colsample_bytree = 0.9,
        n_jobs           = -1,
        random_state     = 4,
        objective        = 'regression',
        reg_alpha        = 1,
        eval_metric      = 'scorer',
        min_child_samples = 15         #叶子节点具有的最小记录数	
        )

In [99]:
y_lgb, y_test_pre = split_score(model_lgb, x_train, y_train, x_test, n=10)

0
1
2
3
4
5
6
7
8
9
10折后的Kappa加权得分为:带补充


In [101]:
y_test_pre = y_test_pre.mean(axis=1)

coe = search_coef(y_lgb, y_train)
best_lgb_coe = coe['x']

print('lgb的最佳系数为{}'.format(best_lgb_coe))
yy_lgb = fix_y(y_lgb, best_lgb_coe)
print('训练集上的二次加权Kappa系数为:', metric(yy_lgb, y_train))


model_lgb.fit(x_train, y_train)
result_lgb_fix = fix_y(y_test_pre, best_lgb_coe)
print('lgb后的分布:',Counter(result_lgb_fix))
print('融合后的二次加权Kappa系数为', metric(result_lgb_fix, y_test))
print('y_test的真实分布为',Counter(y_test))

lgb的最佳系数为[1.5944167  2.118802   2.53260796 2.86564585]
训练集上的二次加权Kappa系数为: 0.44894040647451317
lgb后的分布: Counter({2.0: 832, 4.0: 603, 1.0: 581, 3.0: 503, 0.0: 47})
融合后的二次加权Kappa系数为 0.4374513891575782
y_test的真实分布为 Counter({4: 709, 2: 683, 1: 564, 3: 537, 0: 73})


In [29]:
#参数重要性
model_lgb.feature_importances_
a = pd.DataFrame(model_lgb.feature_importances_, index=x_train.columns)
a.sort_values(by=0, ascending=False)

,0
Breed1,695
RescuerID_count,503
Age,475
IMG_SVD_3,404
State,309
annots_score_MEAN,277
Breed2,272
IMG_SVD_6,229
height_mean,210
Fee,190


# XGB

In [259]:
from xgboost.sklearn import XGBRegressor

model_xgb = XGBRegressor(
    learning_rate    = 0.1,
    n_estimatores    = 100, 
    
    early_stopping_rounds=20,  
    
    max_depth        = 5, 
    min_child_weight = 5,
    
    gamma            = 0,
    
    subsample        =  0.9,
    colsample_bytree = 0.7,
        
    reg_alpha        = 1,
#    reg_lambda       = 0.1,
    nthread      = -1)
#    objective        = 'regression',    
#    eval_metric      = 'scorer')      

model_xgb.fit(x_train.values, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.7, early_stopping_rounds=20, gamma=0,
       learning_rate=0.1, max_delta_step=0, max_depth=5,
       min_child_weight=5, missing=None, n_estimatores=100,
       n_estimators=100, n_jobs=1, nthread=-1, objective='reg:linear',
       random_state=0, reg_alpha=1, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=0.9)

In [262]:
y_xgb = split_score(model_xgb, x_train, y_train, x_test)

coe = search_coef(y_xgb, y_train)
best_xgb_coe = coe['x']
print('xgb的最佳系数为{}'.format(best_xgb_coe))

model_xgb.fit(x_train.values, y_train)
result_xgb = model_xgb.predict(x_test.values)
result_xgb_fix = fix_y(result_xgb, best_xgb_coe)
print('xgb后的分布:',Counter(result_xgb_fix))
print('xgb后的二次加权Kappa系数为', metric(result_xgb_fix, y_test))
print('y_test的真实分布为',Counter(y_test))

ValueError: DataFrame.dtypes for data must be int, float or bool.
                Did not expect the data types in fields Breed1, Breed2, Gender, Color1, Color2, Color3, Vaccinated, Dewormed, Sterilized, Health, Quantity, State, IsFree, Year, Age_qcut, IsRare1, IsRare2, Is_COMMON, Photo_cut, Is_rare_color1, Is_rare_color2, Is_less_than_2month, RescuerID_count_cut, HasSecondBreed, True_Pure, Is_Pure_Dog

In [389]:
#参数重要性
model_xgb.feature_importances_
a = pd.DataFrame(model_xgb.feature_importances_, index=x_train.columns)
a.sort_values(by=0, ascending=False)

,0
Age,0.015806
RescuerID_count,0.013347
annots_score_MEAN,0.010186
Breed1,0.010186
SVD_Description_49,0.008430
annots_score_SUM,0.007727
35,0.007727
9,0.007727
Quantity,0.007025
SVD_Description_33,0.006674


# 后处理

In [390]:
best_coe  = (best_lgb_coe + best_xgb_coe) / 2
best_coe = best_xgb_coe

result = (result_lgb + result_xgb) / 2
result_fix = fix_y(result, best_coe)

print('融合后的二次加权Kappa系数为', metric(result_fix, y_test))



融合后的二次加权Kappa系数为 0.38851531123422645


In [391]:
y_lgb = model_lgb.predict(x_train)

y_xgb = model_xgb.predict(x_train)
y_rf  = model_rf.predict(x_train)


NameError: name 'model_rf' is not defined

In [ ]:
y = (result_lgb + result_xgb + result_rf)/3
result = fix_y(y, best_coe)

print('融合后的分布:',Counter(result))
print('融合后的二次加权Kappa系数为', metric(result, y_test))
print('y_test的真实分布为',Counter(y_test))


In [ ]:
# 特征重要性
model_lgb.feature_importances_
a = pd.DataFrame(model_lgb.feature_importances_, index=x_train.columns)
a.sort_values(by=0, ascending=False)